MERGE FILTERED UMLS DE CUI BACK WITH FITBIR DATA ELEMENT REPORT


<br>
This script will incorporate the new UMLS Data Element (DE) concept attributes into a data element report so that it can be imported into BRICS.<br>
DESCRIPTION:<br>
    <br>
(1): Takes DE_step1_XYZ_mm_UMLS_keepCol.xlsx file and filters rows that are manually marked in the KEEP column<br>
(2): Merges reduced UMLS DE CUI set back with original dataElementExport.csv file<br>
(3): Creates summary file so user can check if any DE cuis are missing<br>
NEW COLUMNS IN DATA ELEMENT IMPORT: <br>
    <br>
1) data element concept names: 3 concept names max per CDE, e.g. "Traumatic Brain Injury | neuropathology disease| Neuropathology<br>
2) data element concept identifiers 3 concepts (name+CUI) max per CDE, e.g., "Traumatic Brain Injury (C0876926)| neuropathology disease (C1540677)| Neuropathology (C0876934)<br>
3) data element terminology sources: e.g. UMLS, NCIt, SNOMED, HL7, LOINC, RxNORM, etc.<br>
4) *permissible value concept names: e.g. "Traumatic Brain Injury | neuropathology disease| Neuropathology <br>
5) *permissible value concept identifiers: each PV must be assigned only one pair "concept names and CUI", e.g., No(C1298908)|Not Assessed(C3846720)|Unknown(C0439673)|Yes (indicator)(C1705108)<br>
6) *permissible value terminology sources: the source for data element concept(s), e.g. UMLS, NCIt, SNOMED, HL7, LOINC, RxNORM, etc.<br>
* these columns will only be included in the ouput if the PV scripts have already been run and the generated columns are <br>
part of the DE_report tab in DE_step1_XYZ_mm_UMLS_keepCol.xlsx<br>


LOAD PACKAGES

In [14]:
import cui_mapping_functions as cmf # contains custom functions for loading/processing FITBIR csv datafiles
from check_missing_cuis import count_sep, count_DE_CUIs_missing, count_CUIs_NA, count_CUIs_per_sep, returnFlaggedCUIs
from functools import partial
import importlib
import pandas as pd
import numpy as np
import os
import tkinter as tk  # for dialog boxes
from tkinter import filedialog
root = tk.Tk()
root.withdraw()
importlib.reload(cmf)

<module 'cui_mapping_functions' from 'C:\\Users\\armengolkm\\Documents\\BRICS\\Sourcetree_BRICS\\Operations\\UMLS Coding\\UMLS-CUI_DE-PV_pipeline_ipynb_Version-1.5_2022-12-06\\cui_mapping_functions.py'>

INITIALIZATION

In [15]:
verify_cuis = 1  # 0 to skip running check_missing_cuis.py functions
sourceTerm = 'UMLS' # currently this overrides terminology source values and sets everything to UMLS

LOAD "DE_step1_XYZ_mm_UMLS_keepCol.xlsx" file

In [16]:
fp_xlsx_file = filedialog.askopenfilename(
    title='Choose xlsx file "DE_step1_XYZ_mm_UMLS_keepCol"')  # file with DE rows marked for keeping
sheet1 = 'DEs_UMLS_MetaMap'
sheet2 = 'DE_report'
df_DEs_UMLS = pd.read_excel(fp_xlsx_file, sheet_name=sheet1, header=0)
df_DE_report = pd.read_excel(fp_xlsx_file, sheet_name=sheet2, header=0)

CREATE STEP 2 DIRECTORY

In [17]:
dir_parent = os.path.dirname(os.path.dirname(fp_xlsx_file))
dir_step2 = cmf.create_folder(os.path.join(
    dir_parent, 'DE_Step-2_Filter UMLS CUIs and Merge with DE Report'))

REMOVE UNNEEDED DE UMLS CODES

In [18]:
df_DEs_UMLS['keep'].replace(r'^\s*$', np.nan, regex=True, inplace=True) # makes sure any empty spaces are nan and will be removed in next step
df_DEs_UMLS_filtered = df_DEs_UMLS[df_DEs_UMLS['keep'].notnull()].reset_index(
    drop=True)

In [19]:
if 'sourceTerm' in locals():
    df_DEs_UMLS_filtered.loc[df_DEs_UMLS_filtered['Sources']
                             != 'Not Available', 'Sources'] = sourceTerm

SORT KEEP COLUMN ORDER

In [20]:
df_DEs_UMLS_filtered['keep'] = df_DEs_UMLS_filtered['keep'].astype(str)
df_DEs_UMLS_filtered['keep_num'] = [x[0] for x in df_DEs_UMLS_filtered['keep']]
df_DEs_UMLS_filtered['keep_letter'] = [x[1:] if len(x) > 1 else '' for x in df_DEs_UMLS_filtered['keep']]
df_DEs_UMLS_filtered = df_DEs_UMLS_filtered.sort_values(by=['keep', 'keep_letter'])
df_DEs_UMLS_filtered['keep_concat'] = ['concat' if len(x) > 1 else '' for x in df_DEs_UMLS_filtered['keep']]

CONCATENATE DEs WITH MULTIPLE CUIS

In [21]:
df_DEs_mult = df_DEs_UMLS_filtered.groupby(["variable name", "title", "keep_concat"])[
    ['CandidatePreferred', 'CandidateCUI', 'Sources', 'keep_num']].agg('/'.join)  # aggregate multiple DE CUIs and separate by '/'
df_DEs_mult = df_DEs_mult.sort_values(by=['keep_num']) # sort by keep_num column
df_DEs_mult = df_DEs_mult.groupby(["variable name", "title"])[
    ['CandidatePreferred', 'CandidateCUI', "Sources"]].agg('|'.join)  # aggregate multiple DE CUIs and separate by '/'
# df_sourceTerm_mult = df_DEs_UMLS_filtered.groupby(["variable name", "title"])[['Sources']].agg(
#     '|'.join)  # aggregate multiple source terminologies and separate by '/'
# df_temp0 = pd.merge(df_DEs_mult, df_sourceTerm_mult, on=["variable name"])
df_DEs_mult['Sources'] = df_DEs_mult['Sources'].apply(lambda x: '|'.join([sourceTerm for i in x.split('|')]))
df_temp0 = df_DEs_mult

MERGE UNIQUE DE CODES WITH FITBIR DATA ELEMENT REPORT

In [22]:
df_final = pd.merge(df_temp0, df_DE_report, how="right",
                    left_on=["variable name"],
                    right_on=["variable name"])  # merge DE CUI columns with FITBIR DE report
df_final.drop_duplicates(inplace=True)
    #df_final = df_final.fillna('')

RENAME COLUMNS FOR FITBIR AND REORDER

In [23]:
try:
    df_final.drop(columns = ["data element concept names", "data element concept identifiers", "data element terminology sources"],
                  inplace = True)
except:
    print("Data element concept fields weren't present in the initial data element report file.")
    
df_final.rename(columns={"CandidatePreferred": "data element concept names", "CandidateCUI": "data element concept identifiers",
                         "Sources": "data element terminology sources"}, inplace=True)
col_reorder = ["variable name", "title",
               "data element concept names",  "data element concept identifiers", "data element terminology sources",
               "element type", "definition", "short description", "datatype", "maximum character quantity", "input restriction",
               "minimum value", "maximum value",
               "permissible values", "permissible value descriptions", "permissible value output codes",
               "permissible value concept names", "permissible value concept identifiers", "permissible value terminology sources",
               "unit of measure", "guidelines/instructions", "notes", "preferred question text", "keywords", "references", "historical notes", "see also",
               "effective date", "until date",
               "population.all", "domain.general (for all diseases)", "domain.traumatic brain injury", "domain.Parkinson's disease", "domain.Friedreich's ataxia", "domain.stroke",
               "domain.amyotrophic lateral sclerosis", "domain.Huntington's disease", "domain.multiple sclerosis", "domain.neuromuscular diseases", "domain.myasthenia gravis",
               "domain.spinal muscular atrophy", "domain.Duchenne muscular dystrophy/Becker muscular dystrophy", "domain.congenital muscular dystrophy", "domain.spinal cord injury",
               "domain.headache", "domain.epilepsy",
               "classification.general (for all diseases)", "classification.acute hospitalized", "classification.concussion/mild TBI", "classification.epidemiology", "classification.moderate/severe TBI: rehabilitation",
               "classification.Parkinson's disease", "classification.Friedreich's ataxia", "classification.stroke", "classification.amyotrophic lateral sclerosis", "classification.Huntington's disease",
               "classification.multiple sclerosis", "classification.neuromuscular diseases", "classification.myasthenia gravis", "classification.spinal muscular atrophy",
               "classification.Duchenne muscular dystrophy/Becker muscular dystrophy", "classification.congenital muscular dystrophy", "classification.spinal cord injury", "classification.headache",
               "classification.epilepsy",  "Label(s)", "submitting organization name", "submitting contact name", "submitting contact information",
               "steward organization name", "steward contact name", "steward contact information"]
cols_pv = ["permissible value concept names",
           "permissible value concept identifiers", "permissible value terminology sources"]
if all(col in df_final.columns for col in cols_pv): 
    df_final = df_final[col_reorder] # will reorder columns to align with DE import template. if/else statement if new PV columns aren't present yet
else:
    col_reorder = [col for col in col_reorder if col not in cols_pv]
    df_final = df_final[col_reorder]

Data element concept fields weren't present in the initial data element report file.


SAVE FINALIZED IMPORT TEMPLATE

In [24]:
filename = os.path.basename(fp_xlsx_file).split('_')[2]
fp_step2 = f"{dir_step2}/DE_step2_{filename}_dataElementImport_cuis.csv"
df_final.to_csv(fp_step2, index=False)  # output df_final dataframe to csv

VERIFY NUMBER OF CUIS STEPS

In [25]:
if verify_cuis:

    ## select relevant columns
    df_verify = df_final[['variable name', 'title', 'definition', 'permissible values',
                          'permissible value descriptions', 'permissible value output codes', 'data element concept names',
                          'data element concept identifiers', 'data element terminology sources']].copy()

    ## de verification
    n_des, des = zip(*[(x[0], x[1]) for x in list(map(partial(count_sep,
                     sep="|"), df_verify['data element concept names']))])
    n_missing, idx_missing = zip(*[(x[0], x[1]) for x in list(
        map(count_DE_CUIs_missing, df_verify['data element concept identifiers']))])
    n_NA, idx_NA = zip(*[(x[0], x[1]) for x in list(map(count_CUIs_NA,
                       df_verify['data element concept identifiers']))])
    n_mult, idx_mult_cui = zip(*[(x[0], x[1]) for x in list(
        map(partial(count_CUIs_per_sep, sep1 = "|", sep2 = "/"), df_verify['data element concept identifiers']))])
    ls_de_missing_cui = returnFlaggedCUIs(des, idx_missing)
    ls_de_cui_NA = returnFlaggedCUIs(des, idx_NA)
    ls_de_mult_cui = returnFlaggedCUIs(des, idx_mult_cui)
    n_de_missing_cui = list(
        map(lambda x: len(x) if type(x) is list else 1, ls_de_missing_cui))
    n_de_cui_NA = list(map(lambda x: len(x) if type(x)
                       is list else np.nan, ls_de_cui_NA))
    # n_de_mult_cui = list(map(lambda x: len(x), ls_de_mult_cui))
    # n_de_mult_cui = list(map(lambda x: (len(x) if len(x) > 0 else 1) if type(x) is list else 0, ls_de_mult_cui))
    # n_single_mult_cui = np.array(n_des) - np.array(n_mult) 

    ## add verification output as columns
    df_verify['de missing cui'] = ls_de_missing_cui
    df_verify['total # de missing cui'] = n_de_missing_cui
    df_verify['de cui Not Available'] = ls_de_cui_NA
    df_verify['total # de cuis Not Available'] = n_de_cui_NA
    df_verify['total # of cui terms'] = list(n_des)
    df_verify['total # multi-cui terms'] = list(n_mult)
    df_verify['de with multiple cuis'] = ls_de_mult_cui

In [26]:
    ## write verification file
    fp_step2_verify = f"{dir_step2}/DE_step2_{filename}_check-missing_cui.csv"
    df_verify.to_csv(fp_step2_verify, index=False)